# sentence-transformers日本語版

https://github.com/sonoisa/sentence-transformers

In [0]:
!git clone https://github.com/sonoisa/sentence-transformers
!cd sentence-transformers; pip install -r requirements.txt

In [0]:
#!cd sentence-transformers; python setup.py install

In [0]:
!wget -O sonobe-datasets-sentence-transformers-model.tar "https://www.floydhub.com/api/v1/resources/JLTtbaaK5dprnxoJtUbBbi?content=true&download=true&rename=sonobe-datasets-sentence-transformers-model-2"
!tar -xvf sonobe-datasets-sentence-transformers-model.tar

In [0]:
%cd sentence-transformers

In [0]:
%tensorflow_version 2.x
from sentence_transformers import SentenceTransformer
import numpy as np

model_path = "/content/training_bert_japanese"
model = SentenceTransformer(model_path, show_progress_bar=False)

In [0]:
# 出典: https://qiita.com/sonoisa/items/775ac4c7871ced6ed4c3 で公開されている「いらすとや」さんの画像タイトル抜粋（「のイラスト」「のマーク」「のキャラクター」という文言を削った）
sentences = ["お辞儀をしている男性会社員", "笑い袋", "テクニカルエバンジェリスト（女性）", "戦うAI", "笑う男性（5段階）", 
"漫才師", "お辞儀をしている医者（女性）", "お辞儀をしている薬剤師", "福笑いをしている人", "AIの家族", "コント師", 
"福笑い（女性）", "お辞儀をしている犬", "苦笑いをする女性", "お辞儀をしている医者", "いろいろな漫符", 
"雛人形「仕丁・三人上戸」", "ダンス「踊る男性」", "拍手をしている人", "定年（男性）", "ものまね芸人", "福笑いのおたふく", 
"お辞儀をしている看護師（男性）", "愛想笑い", "福笑い（ひょっとこ）", "成長する人工知能", "苦笑いをする男性", 
"運動会「徒競走・白組」", "人工知能と喧嘩をする人", "人工知能", "ありがた迷惑", "お辞儀をしているクマ", "笑う女性（5段階）", 
"人工知能とメールをする人（男性）", "技術書", "笑いをこらえる人（女性）", "ダンス「踊る女性」", "お辞儀をしている猫", 
"福笑い（男性）", "武器を持つAI", "作曲する人工知能", "縄跳びを飛んでいる女性", "福笑い（おかめ）", "茅の輪くぐり", "表情", 
"AIと仲良くなる人間", "お笑い芸人「漫才師」", "人工知能とメールをする人（女性）", "人工知能と戦う囲碁の棋士", "拍手している女の子", 
"検索する人工知能", "ピースサインを出す人（女性）", "啓示を受けた人（女性）", "仕事をする人工知能", "一輪車に乗る女の子", 
"お辞儀をしているウサギ", "走る猫（笑顔）", "人工知能と戦う将棋の棋士", "遠足「お弁当・男の子・女の子」", "心を持ったAI", 
"プレゼントをもらって喜ぶ女の子", "技術書（バラバラ）", "いろいろな表情の酔っぱらい（男性）", "拍手している人（棒人間）", 
"仕事を奪う人工知能", "文章を書く人工知能", "いろいろな映画の「つづく」", "絵を描く人工知能", "拍手している男の子", "ハリセン", 
"人工知能と仲良くする人たち", "ON AIRランプ", "いろいろな表情の酔っぱらい（女性）", "徹夜明けの笑顔（女性）", 
"徹夜明けの笑顔（男性）", "お辞儀をしている女性会社員", "バンザイをしているお婆さん", "画像認識をするAI", 
"芸人の男の子（将来の夢）", "料理「女性」", "ピコピコハンマー", "鏡を見る人（笑顔の男性）", "笑いをこらえる人（男性）", 
"シンギュラリティ", "人工知能に仕事を任せる人", "スマートスピーカー", "学ぶ人工知能", "人工知能・AI", "英語のアルファベット", 
"お金を見つめてニヤけている男性", "「ありがとう」と言っている人", "定年（女性）", "テクニカルエバンジェリスト（男性）", 
"スタンディングオベーション"]

In [0]:
sentence_vectors = model.encode(sentences)

## 意味が近い文をクラスタリングしてみる

In [0]:
from sklearn.cluster import KMeans

num_clusters = 8
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(sentence_vectors)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(sentences[sentence_id])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

## 意味が近い文を検索してみる

In [0]:
import scipy.spatial

queries = ['暴走したAI', '暴走した人工知能', 'いらすとやさんに感謝', 'つづく']
query_embeddings = model.encode(queries)

closest_n = 5
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_vectors, metric="cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx, distance in results[0:closest_n]:
        print(sentences[idx].strip(), "(Score: %.4f)" % (distance / 2))

## TensorBoardで潜在意味空間を可視化してみる

TensorBoardが起動したら、右上のメニューからPROJECTORを選択してください。
可視化アルゴリズム（TensorBoardの左下ペイン）はUMAPの2D、neighbors（TensorBoardの右ペイン）は10に設定すると見やすいでしょう。

In [0]:
%load_ext tensorboard
import os
logs_base_dir = "runs"
os.makedirs(logs_base_dir, exist_ok=True)

In [0]:
import torch
from torch.utils.tensorboard import SummaryWriter

import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

summary_writer = SummaryWriter()
summary_writer.add_embedding(mat=np.array(sentence_vectors), metadata=sentences)

In [0]:
%tensorboard --logdir {logs_base_dir}